In [1]:
from time import ctime
import time
import os
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import tkinter as tk
import pyttsx3
import speech_recognition as sr
import webbrowser  
import datetime  
import wikipedia 
import google
import pyowm
import re
from tkinter import *
import tkinter as tk


In [2]:
def listen():
    l1.delete("1.0", END)
    l1.insert(END," listening...")
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
    data = ""
    try:
        data = r.recognize_google(audio)
        l1.delete("1.0", END)
        l1.insert(END,"You said: " + str(data))
    except sr.UnknownValueError:
        l1.delete("1.0", END)
        l1.insert(END,"Google Speech Recognition did not understand audio")
    except sr.RequestError as e:
        l1.delete("1.0", END)
        l1.insert(END,"Request Failed; {0}".format(e))
    return data

def respond(data):
    if "how are you" in data:
        listening = True
        l1.delete("1.0", END)
        l1.insert(END,"I am well")

    elif "what time is it" in data:
        listening = True
        l1.delete("1.0", END)
        l1.insert(END,ctime())
        
    elif "search" in data:
        search=listen()
        url='https://google.com/search?q = '+search
        webbrowser.get().open(url)
        l1.delete("1.0", END)
        l1.insert(END,"here i found"+search)
    elif "search location" in data:
        search=listen()
        url='https://google.nl/maps/place/'+location+'/&amp;'
        webbrowser.get().open(url)
        l1.delete("1.0", END)
        l1.insert(END,"here i found"+location)
    elif "stop listening" in data:
        listening = False
        l1.delete("1.0", END)
        l1.insert(END,'Listening stopped')
     

In [3]:

#data
sites={"google":"www.google.com","wiki":"www.wikipedia.org","numpy":"www.numpy.org"}

def tellDay():
    day = datetime.datetime.today().weekday() + 1
    Day_dict = {1: 'Monday', 2: 'Tuesday', 
                3: 'Wednesday', 4: 'Thursday', 
                5: 'Friday', 6: 'Saturday',
                7: 'Sunday'}
      
    if day in Day_dict.keys():
        day_of_the_week = Day_dict[day]
        l1.delete("1.0", END)
        l1.insert(END,"The day is ",day_of_the_week)
  
  
def tellTime():
      
    time = str(datetime.datetime.now())      
    #print(time)
    hour = time[11:13]
    mins = time[14:16]
    l1.delete("1.0", END)
    l1.insert(END,"The time is sir {} Hours and {} Minutes".format(hour,mins))  

#def takeCommand():
#    command=inputtxt.get()
#    return command
#def Hello():
#    l1.insert(END,"Hello!! I am your Personal Assistant, Kevin. How can I help you?")
#    print("Hi!")
    
def Take_query():
    #Hello()
    
    query = inputtxt.get().lower()
        
    if re.search("open",query) or re.search("search",query):
        for key in sites:
            if re.search(key,query):
                l1.delete("1.0", END)
                l1.insert(END,"Succesfully Opened!!")
                webbrowser.open(sites[key])
                break
                
            else:
                continue
                    
        else:
            l1.delete("1.0", END)
            l1.insert(END,"Seems, I don't have this address registered. Please enter name & full address\n")
            l1.insert(END,"Enter Site Name:")
            nm=input("Site Name:").lower()
            ad=input("Site Address:").lower()
            sites[nm]=ad
            l1.delete("1.0", END)
            l1.insert(END,"Successfully Registered!!")
            webbrowser.open(sites[nm])
                
            
                
    elif re.search("which day it is",query):
        tellDay()
        #continue

    elif (re.search("tell",query) or re.search("what",query)) and re.search("time",query):
        tellTime()
        #continue
    elif re.search("what",query) and re.search("your",query) and re.search("name",query):
        l1.delete("1.0", END)
        l1.insert(END,"I am Kevin. Your Personal Assistant.")
        #continue

    elif re.search("what",query) or re.search("who",query):
        l1.delete("1.0", END)
        l1.insert(END,"I'm checking the wikipedia ")
        if re.search("what",query):
            query = query.replace("what", "")
        elif  re.search("who",query):
            query = query.replace("who", "")

        result = wikipedia.summary(query, sentences=4)
        l1.delete("1.0", END)
        l1.insert(END,"According to wikipedia\n")
        l1.insert(END,result)
        
    elif re.search("salary",query):
        obj=salary(query)
        obj.predict1()

    elif re.search("quit",query) or re.search("exit",query):
        l1.delete("1.0", END)
        l1.insert(END,"I will see you soon!!")
        #break


In [4]:
class salary:
    def __init__(self,query):
        self.query=query
    def predict1(self):
        data=pd.read_csv('Hello.csv')
        X=data[:]
        from sklearn import linear_model
        df=linear_model.LinearRegression()
        df.fit(X[['Salary','HouseRent','Medical Bills','Recharges','Food','Cigg or Alcohol','Shopping','Others']],X.Savings)

        save= self.query
        save= save.replace("salary,","")
        sal,HR,MB,rech,Food,CigAl,shop,oth,month= map(int,save.split(","))
        y_pred = df.predict([[sal,HR,MB,rech,Food,CigAl,shop,oth]])

        saving = y_pred[0]
        #default ideal 50(household+grocey):20(savings):30(rest)
        
        if month==0:
            
            if saving <= 0: 
                l1.delete("1.0", END)
                l1.insert(END,"You are not Saving Anything.")
            else:
                l1.delete("1.0", END)
                l1.insert(END,"You will save {:.2f} Rupees.".format(saving))
                if (HR+MB+rech+Food)<=sal*0.5:
                    
                    if (CigAl+shop+oth)<=sal*0.3:
                        
                        if saving>=sal*0.2:
                            l1.insert(END,"You are spending money Ideally.")
                        else:
                            l1.insert(END,"Try to save More.")
                    else:
                        l1.insert(END,"Try to save More on Ciggaretes/Alcohol, Shopping and Others.")
                else:
                    l1.insert(END,"Try to save More on necessities.")
                
                
                
        else:
            if saving <= 0: 
                l1.delete("1.0", END)
                l1.insert(END,"You are not Saving Anything.")
            else: 
                saving = saving*month
                l1.delete("1.0", END)
                l1.insert(END,"You will save {:.2f} Rupees after {} months.".format(saving,month))
                if (HR+MB+rech+Food)<=sal*0.5:
                    
                    if (CigAl+shop+oth)<=sal*0.3:
                        
                        if saving>=sal*0.2:
                            l1.insert(END,"You are spending money Ideally.")
                        else:
                            l1.insert(END,"Try to save More.")
                    else:
                        l1.insert(END,"Try to save More on Ciggaretes/Alcohol, Shopping and Others.")
                else:
                    l1.insert(END,"Try to save More on necessities.")


In [5]:

root = tk.Tk()
root.geometry("1000x1000")
root.configure(background='black')
w2 = Label(root, justify=CENTER, text="Project Hello", fg="Red", bg="Black")
w2.config(font=("Times",30,"bold italic"))
w2.grid(row=1, column=0, columnspan=2, padx=100)
w2 = Label(root, justify=CENTER, text="A Project by The Innovators", fg="blue", bg="Black")
w2.config(font=("Times",30,"bold italic"))
w2.grid(row=2, column=0, columnspan=2, padx=100)
w3 = Label(root, justify=CENTER, text="Hello!! I am your Personal Assistant, Kevin. How can I help you?", fg="blue", bg="Black")
w3.config(font=("Times",15,"bold italic"))
w3.grid(row=3, column=0, columnspan=2, padx=100)

#Name
Name = StringVar()
NameLb = Label(root, text="Name Of The User", fg="Red", bg="Sky Blue")
NameLb.config(font=("Times",15,"bold italic"))
NameLb.grid(row=6, column=0, pady=15, sticky=W)
NameEn = Entry(root, textvariable=Name)
NameEn.grid(row=6, column=1)

#Input 
inputtxt = StringVar()
inputtxtLb = Label(root, text="Enter Here", fg="Red", bg="Sky Blue")
inputtxtLb.config(font=("Times",15,"bold italic"))
inputtxtLb.grid(row=20, column=0, pady=15, sticky=W)
inputtxtEn = Entry(root,font=("Calibri",15),width=70, textvariable=inputtxt)
inputtxtEn.grid(row=20, column=1)
#inputtxt = Label(root, height = 10,width = 25,bg = "light yellow")

#Start Button For Text
start = Button(root, text="Execute", command=Take_query,bg="Blue",fg="white")
start.config(font=("Times",15,"bold italic"))
start.grid(row=20, column=3,padx=10)

#Start Button For Voice
vc = Button(root, text="mic", command=listen,bg="Blue",fg="white")
vc.config(font=("Times",15,"bold italic"))
vc.grid(row=20, column=4,padx=10)


#Output Window
l1 = Text(root, height = 10,width = 70,bg = "light yellow")
l1.config(font=("Times",15,"bold italic"))
l1.grid(row=10, column=1, padx=10)

root.mainloop()